In [1]:
import numpy as np
from itertools import product
from IPython.display import Audio, display

SR = 44100
TUNING = [82.41, 110.00, 146.83, 196.00, 246.94, 329.63]

# Chord Detection with Fast Fourier Transform

### Intro
Modern computer vision programs can track fingers in the 2D plane easily easily, but *struggle to track depth*. This is a problem since the depth of the finger **drastically changes what chord is actually being played.**

When a computer vision model tracks a finger to be overlapping with a string, there are **three** possibilities:
- The finger is pressing down on the string, or "fretting" the string. This results in the note being played.
- The finger is pressing lightly on the string, or "muting" the string. This results in the note being inaudible.
- The finger is hovering over the string. This results in the "open string" being played, which is usually a different note entirely (one of E, A, D, G, or B).

Since computer vision algorithms currently struggle to distinguish between these three possibilities, we present an approach where the audio of the chord is analyzed to determine the actual chord being played.

### Inputs
This notebook takes in 2 inputs, the first of which is the hypothetical input of what a computer vision algorithm sees. This input is in the form [n, n, n, n, n, n], where n can be X to represent a muted note, 0 to represent the open note, or any integer 0 < n < 20 to represent a fretted note.

A list of candidate chords is then created from this input, where each n from 0 < n < 20 is also replaced with either 0 or X.

The second input, also in the form [n, n, n, n, n, n], represents the "actual" chord being played. An audio of this chord is created using the Karplus-Strong algorithm. The audio is matched with the generated audio of each chord in the candidate chord list, and the most likely candidate chord is then identified.

### Audio Analysis
To identify which candidate matches the played sound, we perform a Fast Fourier Transform (FFT) on each synthesized audio clip.
The FFT converts each time-domain waveform into the frequency domain, revealing the harmonic spectrum of the chord. Each chord has a unique combination of dominant frequencies corresponding to its notes.

By comparing the FFT spectra of the actual chord and each candidate chord, the algorithm identifies which candidate’s frequency profile most closely matches the real sound.

### Using this notebook
All inputs are defined in Cell 19.

Modify each `n` in `test_audio = synth_chord_array([n, n, n, n, n, n], dur=2.2, sr=SR)` to form the test chord, or the chord that is being heard.

Modify each `n` in `candidates = enumerate_possible_chords([n, n, n, n, n, n])` to form the input "chord", aka what the CV algorithm detects.

The default setup is an E minor chord being played, but a finger is also detected overalapping with the first fret G string. This is a good real-world example of the applicability of this approach because:
- The depth of the finger mentioned here makes a big difference in the chord being played.
  - If this finger was **fretting** that note, this would form an **E major** chord.
  - If this finger was hovering over that note, this would form an **E minor** chord.
  - If that finger was muting that note, this would form an **E5** chord.
- Transitionaing from a major chord to a minor chord of the same root is quite common in music (see Creep by Radiohead for an example in key C major).
- This specific scenario is common and **not** an unlikely edge case. Guitarists transitioning from an E major to E minor chord will simply lift their pointer finger (first fret G string) slightly and keep their other fingers where they are.


In [2]:
def freq_from_fret(open_freq: float, fret: int) -> float:
    return open_freq * (2 ** (fret / 12))

def karplus_strong(freq: float, dur: float = 2.0, sr: int = SR) -> np.ndarray:
    N = int(sr * dur)
    if freq <= 0:
        return np.zeros(N, dtype=np.float32)

    L = max(2, int(sr / freq))
    buf = np.random.uniform(-1, 1, L).astype(np.float32)
    out = np.zeros(N, dtype=np.float32)

    for i in range(N):
        out[i] = buf[i % L]
        buf[i % L] = 0.996 * 0.5 * (buf[i % L] + buf[(i + 1) % L])

    return out

def synth_chord_array(frets, dur: float = 2.0, sr: int = SR) -> np.ndarray:
    layers = []
    for i, f in enumerate(frets):
        if f == "X":
            continue
        freq = freq_from_fret(TUNING[i], int(f))
        layers.append(karplus_strong(freq, dur=dur, sr=sr))

    if not layers:
        return np.zeros(int(sr * dur), dtype=np.float32)

    mix = np.sum(layers, axis=0).astype(np.float32)
    mix /= (np.max(np.abs(mix)) + 1e-9)
    return mix


In [3]:
def enumerate_possible_chords(
    overlaps,
    allow_open: bool = True,
    allow_mute: bool = True,
    allow_press: bool = True
):
    assert len(overlaps) == 6, "Provide exactly 6 strings (low E to high E)."

    per_string_options = []
    for val in overlaps:
        opts = []
        if val is None:
            if allow_open: opts.append(0)
            if allow_mute: opts.append("X")
        else:
            if allow_press: opts.append(int(val))
            if allow_open:  opts.append(0)
            if allow_mute:  opts.append("X")
        per_string_options.append(opts)

    seen = set()
    chords = []
    for combo in product(*per_string_options):
        if all(c == "X" for c in combo):  # skip all-muted
            continue
        tup = tuple(combo)
        if tup not in seen:
            seen.add(tup)
            chords.append(list(combo))
    return chords

In [4]:
def _frame_audio(audio: np.ndarray, sr: int = SR, win_s: float = 0.046, hop_s: float = 0.023):
    win = int(win_s * sr)
    hop = int(hop_s * sr)
    if win < 32:
        win = 2048
    if hop < 16:
        hop = win // 2

    nfft = 1
    while nfft < win:
        nfft *= 2

    w = np.hanning(win).astype(np.float32)
    frames = []
    for start in range(0, max(1, len(audio) - win), hop):
        seg = audio[start:start + win]
        if len(seg) < win:
            seg = np.pad(seg, (0, win - len(seg)))
        X = np.fft.rfft(seg * w, n=nfft)
        frames.append(np.abs(X))

    if not frames:
        X = np.fft.rfft(np.pad(audio, (0, win - len(audio))), n=nfft)
        frames = [np.abs(X)]

    S = np.stack(frames, axis=0)
    freqs = np.fft.rfftfreq(nfft, d=1 / sr)
    spec = np.median(S, axis=0)
    spec = spec / (np.max(spec) + 1e-9)
    return spec, freqs

def _hz_to_idx(freqs: np.ndarray, hz: float) -> int:
    if hz <= freqs[0]:
        return 0
    if hz >= freqs[-1]:
        return len(freqs) - 1
    return int(np.argmin(np.abs(freqs - hz)))

def _candidate_harmonics_for_string(open_hz: float, fret: int, max_hz: float, max_harm: int = 6):
    f0 = freq_from_fret(open_hz, fret)
    hs = []
    for h in range(1, max_harm + 1):
        fh = f0 * h
        if fh > max_hz:
            break
        hs.append(fh)
    return hs

def _collect_expected_harmonics(frets, freqs: np.ndarray, max_harm: int = 6):
    expected = []
    for s_idx, f in enumerate(frets):
        if f == "X":
            continue
        if isinstance(f, (int, np.integer)):
            expected.extend(
                _candidate_harmonics_for_string(TUNING[s_idx], int(f), freqs[-1], max_harm=max_harm)
            )
    return expected

def _cents_window(hz: float, cents: float = 35):
    r = 2 ** (cents / 1200.0)
    return hz / r, hz * r

def score_chord_against_audio(
    frets,
    test_spec: np.ndarray,
    test_freqs: np.ndarray,
    cents_tol: float = 35,
    max_harm: int = 6
) -> float:
    expected = _collect_expected_harmonics(frets, test_freqs, max_harm=max_harm)
    if not expected:
        return 0.0

    cumsum = np.cumsum(test_spec)
    tot = cumsum[-1] + 1e-9
    score = 0.0

    for fh in expected:
        lo, hi = _cents_window(fh, cents=cents_tol)
        i0 = _hz_to_idx(test_freqs, lo)
        i1 = _hz_to_idx(test_freqs, hi)
        if i1 <= i0:
            i1 = min(i0 + 1, len(test_spec) - 1)
        band_energy = cumsum[i1] - (cumsum[i0 - 1] if i0 > 0 else 0.0)
        score += band_energy

    return float(score / tot)

def rank_chords_by_audio(
    possible_chords,
    test_audio: np.ndarray | None = None,
    test_wav: str | None = None,
    sr: int = SR,
    cents_tol: float = 35,
    max_harm: int = 6,
    top_k: int = 10
):
    assert (test_audio is not None) or (test_wav is not None), "Provide test_audio or test_wav."

    if test_wav is not None:
        import wave
        with wave.open(test_wav, 'rb') as wf:
            assert wf.getnchannels() == 1, "Use mono WAV."
            assert wf.getsampwidth() == 2, "Use 16-bit PCM WAV."
            assert wf.getframerate() == sr, f"WAV must be {sr} Hz."
            frames = wf.readframes(wf.getnframes())
            test_audio = np.frombuffer(frames, dtype=np.int16).astype(np.float32) / 32768.0
    else:
        test_audio = np.asarray(test_audio, dtype=np.float32)

    test_audio = test_audio - np.mean(test_audio)
    peak = np.max(np.abs(test_audio)) + 1e-9
    test_audio = test_audio / peak

    test_spec, test_freqs = _frame_audio(test_audio, sr=sr)
    scored = []
    for ch in possible_chords:
        s = score_chord_against_audio(ch, test_spec, test_freqs, cents_tol=cents_tol, max_harm=max_harm)
        scored.append((ch, s))
    scored.sort(key=lambda x: x[1], reverse=True)
    return scored[:top_k]


## Using the notebook

Modify each `n` in `test_audio = synth_chord_array([n, n, n, n, n, n], dur=2.2, sr=SR)` to form the test chord, or the chord that is being heard.

Modify each `n` in `candidates = enumerate_possible_chords([n, n, n, n, n, n])` to form the input "chord", aka what the CV algorithm detects.

See cell 2 for an explanation of the default setup.


In [5]:
# The actual chord that is being played
test_audio = synth_chord_array([0, 2, 2, 0, 0, 0], dur=2.2, sr=SR)

# What the hypothetical CV algorithm sees/detects:
candidates = enumerate_possible_chords([0, 2, 2, 1, 0, 0])

top = rank_chords_by_audio(candidates, test_audio=test_audio, top_k=5)
for ch, sc in top:
    print(f"{ch} -> {sc:.4f}")

best_chord = top[0][0]
best_audio = synth_chord_array(best_chord, dur=2.2, sr=SR)

print("\nTest Chord:")
display(Audio(test_audio, rate=SR))

print("\nTop Candidate:")
display(Audio(best_audio, rate=SR))


[0, 2, 2, 0, 0, 0] -> 1.1985
[0, 2, 2, 1, 0, 0] -> 1.1832
[0, 2, 0, 0, 0, 0] -> 1.1531
[0, 0, 2, 0, 0, 0] -> 1.1404
[0, 2, 0, 1, 0, 0] -> 1.1379

Test Chord:



Top Candidate:
